### Edgar Moises Hernandez-Gonzalez
#### 29/04/20
#### Clasificacion de imagenes motoras en EEG con CNN-1D + LSTM o CNN-1D + GRU
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import pywt
import cv2
from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [0]:
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A09E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA09E.csv", header=None)

In [5]:
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 1)


In [6]:
n_samples_test = len(y_test) #en caso de no tener el y_test se puede utilizar el x_test

print("n_samples_test :", n_samples_test)

n_samples_test : 288


In [7]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [8]:
#codificar etiquetas con valores de 0 a nClases-1
y_test = LabelEncoder().fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#unir 22 canales
def unir_escalogramas_vertical_resize(data):
  escalas = np.arange(1,129)
  dim = (125, 352) #ancho, alto
  datos = np.zeros((288,352,125))
  temporal = np.zeros((2816,1000))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3') #escalograma CWT
      temporal[j*128:(j+1)*128, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [10]:
x_test = unir_escalogramas_vertical_resize(x_test)

0
100
200


In [11]:
print(x_test.shape)

(288, 352, 125)


In [0]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= 146.0

In [13]:
#convertir de 3D a 5D
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 1, 352, 125, 1)


In [0]:
model = load_model('/content/drive/My Drive/BCI-IV-2a/Modelos/Modelo_CNN2D_LSTM_CWT09.hdf5')

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 1, 350, 123, 32)   320       
_________________________________________________________________
time_distributed_20 (TimeDis (None, 1, 175, 61, 32)    0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 1, 173, 59, 32)    9248      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 1, 86, 29, 32)     0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 1, 84, 27, 32)     9248      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 1, 42, 13, 32)     0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 1, 40, 11, 32)    

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [17]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.9076
accuracy: 0.6597


In [0]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [19]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.5463 


In [20]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[61  6  2  3]
 [ 7 24 19 22]
 [ 5  5 42 20]
 [ 0  3  6 63]]
